In [9]:
import jinja2
from copy import deepcopy
import subprocess

def make(*args):
    return subprocess.check_output(["make", *args]).decode("UTF-8")

In [5]:
templateLoader = jinja2.FileSystemLoader(searchpath="./")
templateEnv = jinja2.Environment(loader=templateLoader)
cpp_template = templateEnv.get_template("avr-AUTOSAR-trampoline-Blink.cpp.jinja2")
oil_template = templateEnv.get_template("avr-AUTOSAR-trampoline-Blink.oil.jinja2")

In [10]:
tasks = list()
task_default = {
    "alarmtime": 5000,
    "cycletime": 1000,
    "priority": 20,
    "stacksize": 128,
    "c_type": "unsigned char"
}

cycletime=100
for pin in range(8, 14):
    task = deepcopy(task_default)
    task["name"]=f"blinkTaskPin{pin}"
    task["pin"]=pin
    task["cycletime"]=cycletime
    tasks.append(task)
    cycletime=cycletime*2
    
with open("avr-AUTOSAR-trampoline-Blink.cpp", "w") as fid:
    print(cpp_template.render(tasks=tasks), file=fid)
with open("avr-AUTOSAR-trampoline-Blink.oil", "w") as fid:
    print(oil_template.render(tasks=tasks), file=fid)
    
make()
make("flash")

'avrdude -carduino -v -V -patmega328p -P /dev/ttyUSB0 -D -Uflash:w:avr-AUTOSAR-trampoline-Blink.elf.hex\n'